Problem 4.1

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

file_path = '/content/drive/My Drive/accidentsFull.csv'

A

In [ ]:
import pandas as pd

data = pd.read_csv(file_path)

data['INJURY'] = data['MAX_SEV_IR'].apply(lambda x: 'Yes' if x in [1, 2] else 'No')

num_injury = data['INJURY'].value_counts()['Yes']
num_no_injury = data['INJURY'].value_counts()['No']
total_accidents = len(data)
proportion_injury = num_injury / total_accidents

print("Proportion of accidents with injuries:", proportion_injury)

if proportion_injury >= 0.5:
    prediction = "Yes"
else:
    prediction = "No"

print("Prediction for a new accident:", prediction)

Proportion of accidents with injuries: 0.5087831590925255
Prediction for a new accident: Yes


B

i

In [ ]:
# Select the first 12 records and the specified columns
subset = data.loc[:11, ['INJURY', 'WEATHER_R', 'TRAF_CON_R']]

print(subset)

   INJURY  WEATHER_R  TRAF_CON_R
0     Yes          1           0
1      No          2           0
2      No          2           1
3      No          1           1
4      No          1           0
5     Yes          2           0
6      No          2           0
7     Yes          1           0
8      No          2           0
9      No          2           0
10     No          2           0
11     No          1           2


In [ ]:
# Create the pivot table
pivot_table = pd.pivot_table(subset, index=['WEATHER_R'], columns=['TRAF_CON_R'], values='INJURY', aggfunc='count', fill_value=0)

print(pivot_table)

TRAF_CON_R  0  1  2
WEATHER_R          
1           3  1  1
2           6  1  0


ii

In [ ]:

count_combinations = data.groupby(['WEATHER_R', 'TRAF_CON_R', 'INJURY']).size().reset_index(name='count')

total_accidents = count_combinations['count'].sum()
num_injury = count_combinations[count_combinations['INJURY'] == 'Yes']['count'].sum()

p_injury = num_injury / total_accidents

count_injury = count_combinations[count_combinations['INJURY'] == 'Yes']
count_combinations['p_wt_given_injury'] = count_injury['count'] / num_injury

count_combinations['p_wt'] = count_combinations['count'] / total_accidents

count_combinations['p_injury_given_wt'] = count_combinations['p_wt_given_injury'] * p_injury / count_combinations['p_wt']

print(count_combinations[['WEATHER_R', 'TRAF_CON_R', 'INJURY', 'p_injury_given_wt']])

    WEATHER_R  TRAF_CON_R INJURY  p_injury_given_wt
0           1           0     No                NaN
1           1           0    Yes                1.0
2           1           1     No                NaN
3           1           1    Yes                1.0
4           1           2     No                NaN
5           1           2    Yes                1.0
6           2           0     No                NaN
7           2           0    Yes                1.0
8           2           1     No                NaN
9           2           1    Yes                1.0
10          2           2     No                NaN
11          2           2    Yes                1.0


iii

In [ ]:
cutoff = 0.5

count_combinations['prediction'] = count_combinations['p_injury_given_wt'] >= cutoff

print(count_combinations[['WEATHER_R', 'TRAF_CON_R', 'INJURY', 'p_injury_given_wt', 'prediction']])


    WEATHER_R  TRAF_CON_R INJURY  p_injury_given_wt  prediction
0           1           0     No                NaN       False
1           1           0    Yes                1.0        True
2           1           1     No                NaN       False
3           1           1    Yes                1.0        True
4           1           2     No                NaN       False
5           1           2    Yes                1.0        True
6           2           0     No                NaN       False
7           2           0    Yes                1.0        True
8           2           1     No                NaN       False
9           2           1    Yes                1.0        True
10          2           2     No                NaN       False
11          2           2    Yes                1.0        True


iv

In [ ]:
p_weather1_traf1_given_injury_yes = len(data[(data['WEATHER_R'] == 1) & (data['TRAF_CON_R'] == 1) & (data['INJURY'] == 'Yes')]) / len(data[data['INJURY'] == 'Yes'])
# P(INJURY = Yes)
p_injury_yes = len(data[data['INJURY'] == 'Yes']) / len(data)
# P(WEATHER_R = 1, TRAF_CON_R = 1)
p_weather1_traf1 = len(data[(data['WEATHER_R'] == 1) & (data['TRAF_CON_R'] == 1)]) / len(data)

# Compute the naive Bayes conditional probability
p_injury_yes_given_weather1_traf1 = (p_weather1_traf1_given_injury_yes * p_injury_yes) / p_weather1_traf1

# Print the result
print("Naive Bayes conditional probability of an injury given WEATHER_R = 1 and TRAF_CON_R = 1:", p_injury_yes_given_weather1_traf1)

Naive Bayes conditional probability of an injury given WEATHER_R = 1 and TRAF_CON_R = 1: 0.553423927524647


v

In [ ]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
subset_encoded = subset.apply(le.fit_transform)

X = subset_encoded[['WEATHER_R', 'TRAF_CON_R']]
y = subset_encoded['INJURY']

nb_classifier = CategoricalNB()
nb_classifier.fit(X, y)
predicted_probabilities = nb_classifier.predict_proba(X)
predicted_classes = nb_classifier.predict(X)

print("Predicted probabilities:")
print(predicted_probabilities)
print("Predicted classes:")
print(predicted_classes)

print("Exact Bayes classification:")
print(count_combinations[['WEATHER_R', 'TRAF_CON_R', 'INJURY', 'p_injury_given_wt', 'prediction']])


Predicted probabilities:
[[0.61403509 0.38596491]
 [0.80680571 0.19319429]
 [0.87743733 0.12256267]
 [0.73170732 0.26829268]
 [0.61403509 0.38596491]
 [0.80680571 0.19319429]
 [0.80680571 0.19319429]
 [0.61403509 0.38596491]
 [0.80680571 0.19319429]
 [0.80680571 0.19319429]
 [0.80680571 0.19319429]
 [0.64516129 0.35483871]]
Predicted classes:
[0 0 0 0 0 0 0 0 0 0 0 0]
Exact Bayes classification:
    WEATHER_R  TRAF_CON_R INJURY  p_injury_given_wt  prediction
0           1           0     No                NaN       False
1           1           0    Yes                1.0        True
2           1           1     No                NaN       False
3           1           1    Yes                1.0        True
4           1           2     No                NaN       False
5           1           2    Yes                1.0        True
6           2           0     No                NaN       False
7           2           0    Yes                1.0        True
8           2           1

In [ ]:
result_classifications_equivalent = all(predicted_classes == count_combinations['prediction'].values[:12])

result_ordering_equivalent = all(predicted_classes == count_combinations['prediction'].values[:12])

print("Are the resulting classifications equivalent?", result_classifications_equivalent)
print("Is the ranking (ordering) of observations equivalent?", result_ordering_equivalent)


Are the resulting classifications equivalent? False
Is the ranking (ordering) of observations equivalent? False


C



i

In [ ]:
X = data[['WEATHER_R', 'TRAF_CON_R']]  # Predictors
y = data['INJURY']  # Response variable

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)

# Print the shapes of the training and validation sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)

Training set shape: (25309, 2) (25309,)
Validation set shape: (16874, 2) (16874,)


ii

In [ ]:

X_train = data[['WEATHER_R',  'WRK_ZONE']]
y_train = data['INJURY']

nb_classifier = CategoricalNB()
nb_classifier.fit(X_train, y_train)

y_pred_train = nb_classifier.predict(X_train)

conf_matrix_train = confusion_matrix(y_train, y_pred_train)

print("Confusion Matrix (Training Set):\n", conf_matrix_train)

print("Confusion matrix is ", confusion_matrix)


Confusion Matrix (Training Set):
 [[ 3743 16978]
 [ 3146 18316]]
Confusion matrix is  <function confusion_matrix at 0x7fcc1f07e560>


iii

In [ ]:
X_val = data.loc[X_val.index, ['WEATHER_R', 'WRK_ZONE']]
y_pred_val = nb_classifier.predict(X_val)

accuracy = accuracy_score(y_val, y_pred_val)

overall_error = 1 - accuracy

print("Overall Error for the Validation Set:", overall_error)

Overall Error for the Validation Set: 0.48032475998577695


iv

In [ ]:

class_counts = y_val.value_counts()
majority_class = class_counts.idxmax()
naive_error = 1 - class_counts[majority_class] / len(y_val)
percent_improvement = (naive_error - overall_error) / naive_error * 100

print("Percent Improvement relative to the naive rule:", percent_improvement)


Percent Improvement relative to the naive rule: 2.0307022845400637


v

In [ ]:
pivot_table = data.pivot_table(index='SPD_LIM', columns='INJURY', aggfunc='size', fill_value=0)
conditional_probabilities = pivot_table.div(pivot_table.sum(axis=1), axis=0)

print("Pivot Table with Conditional Probabilities:")
print(conditional_probabilities)

Pivot Table with Conditional Probabilities:
INJURY         No       Yes
SPD_LIM                    
5        0.333333  0.666667
10       0.500000  0.500000
15       0.508197  0.491803
20       0.633466  0.366534
25       0.533888  0.466112
30       0.486406  0.513594
35       0.467627  0.532373
40       0.459572  0.540428
45       0.491878  0.508122
50       0.506907  0.493093
55       0.501365  0.498635
60       0.438480  0.561520
65       0.504972  0.495028
70       0.562586  0.437414
75       0.445230  0.554770
